In [119]:
import json
import random_color 
import copy
import custom_functions
pfile="predicates.json";
apfile="animation_profile.json"
f = open(pfile)
content = f.read()
predicates=json.loads(content)

f1=open(apfile)
content1=f1.read()
animation_profile=json.loads(content1)

In [120]:
objectList=copy.deepcopy(predicates["objects"])
stages=copy.deepcopy(predicates["stages"])
predicates_rules=animation_profile["predicates_rules"]

In [121]:
def initialObjects(objectList,animation_profile):
    unsolvedObjects={}
    for objectname in objectList:
        unsolvedObjects[objectname]={}
        objType=animation_profile["objects"]["default"]
        #update the value for each 
        for objproperty in animation_profile["shape"][objType]:
            value=animation_profile["shape"][objType][objproperty]
            if value is not False:
                if value =="randomcolor":
                    unsolvedObjects[objectname][objproperty]=random_color.get_random_color()
                    continue
                unsolvedObjects[objectname][objproperty]=value
            else:
                unsolvedObjects[objectname][objproperty]=False
        unsolvedObjects[objectname]["name"]=objectname
    return unsolvedObjects

In [122]:
objectsDic=initialObjects(objectList,animation_profile)

In [123]:
space = custom_functions.init_space(len(objectList))

In [124]:
def check_rule_complete(predicate,objectsDic,predicates_rules):
    predicatename=predicate["name"]
    objectnamelist=predicate["objectNames"]
    predicate_rule=predicates_rules[predicatename]
    for rulename in predicate_rule["rules"]:
        rule=predicate_rule[rulename]
        for key in rule:
            #0 is on the left side of equation
            if key != "value" and key !="0":
                property_check=rule[key]
                objectname=objectnamelist[int(key)]
                if objectsDic[objectname][property_check] is False:
                    return False
    return True

In [125]:
def applypredicates(predicate,objectsDic,predicates_rules):
    pname=predicate["name"]
    objects=predicate["objectNames"]
    left=objects[0]
    for rulename in predicates_rules[pname]["rules"]:
        propertyname=predicates_rules[pname][rulename]["0"]
        value=predicates_rules[pname][rulename]["value"]
        rule=predicates_rules[pname][rulename]
        if "function" in value:
            if value["function"]=="distributex":
                objectsDic[left][propertyname]=custom_functions.distributex(left,space,20,80,False)
        elif "equal" in value:
            right_value=value["equal"]
            if right_value in rule:
                right_proterpy=rule[right_value]
                right_object=objects[int(right_value)]
                objectsDic[left][propertyname]=objectsDic[right_object][right_proterpy]
            else:
                objectsDic[left][propertyname]=right_value
                
        elif "add" in value:
            rightvalue=0
            for additem in value["add"]:
                if additem in rule:
                    right_property=rule[additem]
                    right_object=objects[int(additem)]
                    addvalue=objectsDic[right_object][right_property]
                    rightvalue+=addvalue
                else:
                    rightvalue+=additem
            objectsDic[left][propertyname]=rightvalue

In [126]:
def solvepredicates(predicates,objectsDic,predicates_rules):
    while (len(predicates)>0):
        predicate=predicates.pop(0)
        if predicate["name"] not in predicates_rules:
            continue
        if check_rule_complete(predicate,objectsDic,predicates_rules):
            applypredicates(predicate,objectsDic,predicates_rules)
        else:
            if len(predicates)==0:
                return False
            predicates.append(predicate)
    return True

In [127]:
def solveAllStages(stages,objectsDic,predicates_rules):
    result={}
    result["visualStages"]=[]
    for stage in stages:
        stageDic={}
        objectDicCopy=copy.deepcopy(objectsDic)
        predicates=stage["items"]
        solvepredicates(predicates,objectDicCopy,predicates_rules)
        stageDic["visualSprites"]=objectDicCopy
        result["visualStages"].append(stageDic)
    return result

In [128]:
result=solveAllStages(stages,objectsDic,predicates_rules)

In [129]:
def transfer(oneStep,initialobjects):
    # list that stores object name
    finalFile = {}
    temp = []
    # panel size
    panelWidth = 500
    panelHeight = 550
    # dict that stores new position info
    dictX = {"minX":"",
            "maxX":"",
            "minY":"",
            "maxY":""}
    # new position info
    minX = 0.0
    maxX = 0.0
    minY = 0.0
    maxY = 0.0
    # generate new json file
    for i,object in enumerate(initialobjects):
        temp.append(oneStep[object])
        # get all information about position
        xNum = oneStep[object]["x"]
        yNum = oneStep[object]["y"]
        width = oneStep[object]["width"]
        height = oneStep[object]["height"]
        # transfer the position info into position needed in Unity
        minX = xNum/panelWidth
        maxX = (xNum + width)/panelWidth
        minY = yNum/panelHeight
        maxY = (yNum + height)/panelHeight
        dictX["minX"] = round(minX,3)
        dictX["maxX"] = round(maxX,3)
        dictX["minY"] = round(minY,3)
        dictX["maxY"] = round(maxY,3)
        # update the old dict
        temp[i].update(dictX)
        finalFile["visualSprites"] = temp
    return finalFile

final = {"visualStages":[]}
oneStep = {}
spriteList = []
lists = result["visualStages"]
for i,step in enumerate(lists):
    oneStep = lists[i]["visualSprites"]
    spriteList.append(transfer(oneStep,objectList))
    final["visualStages"] = spriteList

with open('visualisation.json', 'w') as outfile:
    json.dump(final, outfile)